In [1]:
#!/usr/bin/env python
from __future__ import division, print_function, absolute_import
import numpy as np
import h5py
import simdna.synthetic as synthetic
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    roc_curve, precision_recall_curve)
import re
from collections import OrderedDict, defaultdict

def load_motif_matches(motif_match_file, doprint=False):
        """
        Loads a homer motif match file into an ordered dictionary with key as se
quence name
        and value as list of dictionaries each containing the keys - motif, sequ
ence,
        begin (0-indexed inclusive begin index of motif), end (0-indexed exclusi
ve end index),
        strand (+ or -), seqval. Each dictionary
        represents one motif match on that sequence
        """
        motif_matches = OrderedDict()
        fp = open(motif_match_file, "r")
        if doprint:
                print("#Loading " + motif_match_file + " ...")
        numlines = 0
        for line in fp:
                match = re.match("((\w|\-)+)\s+((\w|\:|\-)+)\s+(\d+)\s+(\d+)\s+(\+|\-)\s+.+\s+(\w+)$", line)
                if match:
                        numlines = numlines + 1
                        motif = match.group(1)
                        sequence = match.group(3)
                        begin = int(match.group(5))
                        end = int(match.group(6))
                        strand = match.group(7)
                        seqval = match.group(8)
                        entry = dict()
                        entry['motif'] = motif
                        entry['sequence'] = sequence
                        entry['begin'] = begin-1 # Homer motif match file is 1 indexed, convert to 0
                        entry['end'] = end # Homer motif match file is 1 indexed AND inclusive, convert to 0 and exclusive
                        entry['strand'] = strand
                        entry['seqval'] = seqval
                        if sequence not in motif_matches:
                                motif_matches[sequence] = list()
                        motif_matches[sequence].append(entry)
        fp.close()
        if doprint:
                print("#Loaded " + str(numlines) + " motif matches in " + str(len(motif_matches.keys())) + " sequences")
        return motif_matches

In [2]:
def rename(label):
    match=re.match('.*_(chr.*)$',label)
    if match:
        return match.group(1)
    else:
        return ""

In [3]:
def get_relevant_labels_in_order_of_scores(labels, motif_matches):
    relevant_labels_list=[]
    relevant_indices_list=[]
    sequence_index=0
    positive_labels=[]
    for label in motif_matches.keys():
        positive_labels.append(label)
    positive_labels_set = set(positive_labels)
    print("Motif matches sequences are " + str(len(positive_labels_set)))
    print("Supplied labels are " + str(len(labels)))
    for sequence_label in labels:
        if sequence_label in positive_labels_set:
            if sequence_index==731:
                print(sequence_index)
            relevant_indices_list.append(sequence_index)
            relevant_labels_list.append(sequence_label)
        else:
            print("Did not find this label in motif matches: " + sequence_label)
        sequence_index=sequence_index+1
    print (len(relevant_indices_list))
    print (len(relevant_labels_list))
    return relevant_indices_list, relevant_labels_list

In [4]:
def get_relevant_scores(relevant_indices_list, scores, seq_len=400):
    relevant_scores=np.zeros((len(relevant_indices_list),seq_len))
    index=0
    for scores_index in relevant_indices_list:
        relevant_scores[index]=scores[scores_index]
        index=index+1
    return relevant_scores

In [5]:
scores_file='/users/eprakash/projects/benchmarking/newdata/A549/models/deepseabeluga/results/A549.deepseabeluga.scores.5Ksubsample.h5'
h5f = h5py.File(scores_file,'r')
initial_labels=np.array(h5f.get("labels"))
size=len(initial_labels)
labels=np.empty(size,dtype=object)
#fpointer = open("/users/eprakash/projects/benchmarking/newdata/K562/K562_pos_labels_truncated.txt", "w")
for index in range(0, size):
    labels[index]=rename(initial_labels[index])
    #fpointer.write(str(labels[index]) + "\n")
#Sanity check
#fpointer.close()
print (labels.shape)

original_grad_times_input_scores=np.array(h5f.get("grad_times_inp"))
original_rescale_all_scores=np.array(h5f.get("rescale_all_layers_multiref_10"))
original_rescale_conv_scores=np.array(h5f.get("rescale_conv_revealcancel_fc_multiref_10"))

ig_scores=np.array(h5f.get("integrated_gradients10_multiref_10"))
h5f.close()

seq_len=original_grad_times_input_scores.shape[1]
motif_matches=load_motif_matches('/users/eprakash/projects/benchmarking/newdata/A549/A549.motif.matches.txt', True)
initial_seq_ids_of_interest = motif_matches.keys()
seq_ids_of_interest=[]
for id in initial_seq_ids_of_interest:
    seq_ids_of_interest.append(rename(id))
print(len(seq_ids_of_interest))

(5000,)
#Loading /users/eprakash/projects/benchmarking/newdata/A549/A549.motif.matches.txt ...
#Loaded 4599731 motif matches in 143217 sequences
143217


In [6]:
seq_ids_of_interest_set = set(seq_ids_of_interest)
relevant_indices_list, relevant_labels_list=get_relevant_labels_in_order_of_scores(labels, motif_matches)
#Changed later: reducing seq_ids_of_interest to those which are common to motif matches and labels
seq_ids_of_interest = relevant_labels_list
seq_ids_of_interest_set = set(relevant_labels_list)
ig_h5f=h5py.File('/users/eprakash/projects/benchmarking/newdata/A549/models/deepseabeluga/results/top1kposlabels_A549_1kb','r')
ig_seq_ids_of_interest=list(ig_h5f.get("labels"))
grad_times_input_scores=get_relevant_scores(relevant_indices_list, original_grad_times_input_scores, seq_len)
rescale_all_scores=get_relevant_scores(relevant_indices_list, original_rescale_all_scores, seq_len)
rescale_conv_scores=get_relevant_scores(relevant_indices_list, original_rescale_conv_scores, seq_len)
#reveal_cancel_scores=get_relevant_scores(relevant_indices_list, original_reveal_cancel_scores, seq_len)
#guided_backprop_scores=get_relevant_scores(relevant_indices_list, original_guided_backprop_scores, seq_len)

print (rescale_conv_scores.shape)

Motif matches sequences are 143217
Supplied labels are 5000
731
5000
5000
(5000, 400)


In [7]:
for index in range(0,len(ig_seq_ids_of_interest)):
    ig_seq_ids_of_interest[index]=rename(ig_seq_ids_of_interest[index])

In [8]:
import re
method_to_saved_scores = OrderedDict([
    ('grad_times_input', grad_times_input_scores),
    ('rescale_all', rescale_all_scores),
    ('rescale_conv', rescale_conv_scores),
    #('guided_backprop', guided_backprop_scores),
    #('reveal_cancel', reveal_cancel_scores),
    ('integrated_gradients', ig_scores)

])
method_to_seq_id_to_scores = {}
for method_name in method_to_saved_scores:
    scores = method_to_saved_scores[method_name]
    if (method_name=='integrated_gradients'):
        assert(len(ig_seq_ids_of_interest)==len(scores))
        seq_id_to_scores = dict(zip(ig_seq_ids_of_interest,scores))
    else:
        print(len(seq_ids_of_interest))
        print(len(scores))
        assert(len(seq_ids_of_interest)==len(scores))
        seq_id_to_scores = dict(zip(relevant_labels_list,scores))
    method_to_seq_id_to_scores[method_name] = seq_id_to_scores


#covered_positions marks a position as a 1 if it overlaps
# an embedded motif
seq_id_to_covered_positions = {}
motif_id_to_hit_locations = defaultdict(list)
motif_id_to_motif_length = {}
#For each motif, we need a mapping from the motif
# to the positives locations
for seq_id in motif_matches:
    original_seq_id=seq_id
    #seq_id=rename(seq_id)
    if (method_name=='integrated_gradients'):
        if (seq_id in ig_seq_ids_of_interest):
            embedded_positions = np.zeros(seq_len)
            for embedding in motif_matches[original_seq_id]:
                the_seq=embedding['sequence']
                motif_start_loc = embedding['begin'] 
                motif_end_loc = embedding['end']
                motif_len = motif_end_loc-motif_start_loc
                embedded_positions[motif_start_loc:motif_end_loc] = 1.0
                motif_name = re.match('(\d+)-(\w+)-(\d+)',embedding['motif']).group(2)
                if (motif_name in motif_id_to_motif_length):
                    assert (motif_id_to_motif_length[motif_name]==motif_len)
                else:
                    motif_id_to_motif_length[motif_name] = motif_len
                motif_id_to_hit_locations[motif_name].append(
                    (seq_id,motif_start_loc))
            seq_id_to_covered_positions[seq_id] = embedded_positions 
    else:
        if (seq_id in seq_ids_of_interest_set):
            embedded_positions = np.zeros(seq_len)
            for embedding in motif_matches[original_seq_id]:
                the_seq=embedding['sequence']
                motif_start_loc = embedding['begin'] 
                motif_end_loc = embedding['end']
                motif_len = motif_end_loc-motif_start_loc
                embedded_positions[motif_start_loc:motif_end_loc] = 1.0
                motif_name = re.match('(\d+)-(\w+)-(\d+)',embedding['motif']).group(2)
                if (motif_name in motif_id_to_motif_length):
                    assert (motif_id_to_motif_length[motif_name]==motif_len)
                else:
                    motif_id_to_motif_length[motif_name] = motif_len
                motif_id_to_hit_locations[motif_name].append(
                    (seq_id,motif_start_loc))
        #print(embedded_positions)
            seq_id_to_covered_positions[seq_id] = embedded_positions 

total_motif_bases = 0
total_bases = 0
for (seqname, arr) in seq_id_to_covered_positions.items():
    total_motif_bases = total_motif_bases + np.sum(arr)
    total_bases = total_bases +  len(arr)
print("Motif positions: " + str(total_motif_bases) + ", total positions: " + str(total_bases))

#find windows of a given length that do not overlap any motif
motif_len_to_negatives = defaultdict(list)                            
for motif_len in set(motif_id_to_motif_length.values()):
    for seq_id,covered_positions in seq_id_to_covered_positions.items():
        cumsum = np.array([0]+list(np.cumsum(covered_positions)))
        window_sums = cumsum[motif_len:]-cumsum[0:-motif_len]
        null_windows = [(seq_id,x) for x in
                        np.nonzero(window_sums==0)[0]]
        motif_len_to_negatives[motif_len].extend(null_windows)

5000
5000
5000
5000
5000
5000
Motif positions: 178369.0, total positions: 400000


In [9]:
motif_id_to_pos_locs={}
for motif_id in motif_id_to_hit_locations:
    motif_len = motif_id_to_motif_length[motif_id]
    num_pos_locs = len(motif_id_to_hit_locations[motif_id])
    #num_neg_locs = len(motif_len_to_negatives[motif_len])
    motif_id_to_pos_locs.update({motif_id:float(num_pos_locs)})
print(len(motif_id_to_pos_locs))
top_5_motif_ids=sorted(motif_id_to_pos_locs, key=lambda x: motif_id_to_pos_locs[x])
#['TGAATGAATGAA', 'GGGGCGGGGC', 'AGAGGAAGTG', 'AAAAAAAAAAAA', 'AAAAAAAAAA', 'CTTCCTCT', 'GCCACTGCAC', 'AAAAAAAA', 'GGGTGGGG'] 
#top_5_motif_ids=sorted(motif_id_to_hit_locations, key=lambda x: len(motif_id_to_hit_locations[x]))[-5:]
#rescale_all_scores=np.random.rand(rescale_all_scores.shape[0], rescale_all_scores.shape[1])

75


In [10]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 15}

matplotlib.rc('font', **font)

method_name_to_plot_style = {
    'grad_times_input': {'color':'green'},
    'rescale_all': {'color':'orange'},
    'rescale_conv': {'color':'purple'},
    'reveal_cancel': {'color': 'red'},
    'guided_backprop': {'color': 'yellow'},
    'integrated_gradients': {'color': 'blue'}
}

method_name_nicename = {
    'grad_times_input': 'grad_times_input',
    'rescale_all': 'rescale_all',
    'rescale_conv': 'rescale_conv',
    'reveal_cancel': 'reveal_cancel',
    'guided_backprop': 'guided_backprop',
    'integrated_gradients': 'integrated_gradients'
}
motif_info={}
print("Number of motifs is " + str(len(motif_id_to_motif_length.keys())))
for motif_id in sorted(top_5_motif_ids):
    
    for methods_to_plot in [['integrated_gradients']]:
    #[['grad_times_input', 'rescale_all', 'rescale_conv', 'reveal_cancel', 'guided_backprop', 'integrated_gradients']]:
        
        #print("On motif",motif_id)
        motif_len = motif_id_to_motif_length[motif_id]
        pos_locs = motif_id_to_hit_locations[motif_id]
        neg_locs = motif_len_to_negatives[motif_len]
        #print("num pos",len(pos_locs))
        #print("num neg",len(neg_locs))
        all_locs = list(pos_locs)+list(neg_locs)
        #print(len(motif_matches.items()))
        loc_labels = [1 for x in pos_locs]+[0 for x in neg_locs]

        #f,axarr = plt.subplots(1,2, figsize=(20,5))
        #plt.subplot(122)
        #plt.title(motif_id+' auROC Curves')
        #plt.plot([0, 1], [0, 1], color='black', linestyle='--')
        #plt.xlim([0, 1])
        #plt.ylim([0, 1])
        #plt.ylabel('True Positive Rate')
        #plt.xlabel('False Positive Rate')

        pos_frac = float(len(pos_locs))/(len(pos_locs)+len(neg_locs))
        #plt.subplot(121)
        #plt.title(motif_id+' auPRC Curves')
        #plt.plot([0, 1], [pos_frac, pos_frac], color='black', linestyle='--')
        #plt.xlim([0, 1])
        #plt.ylim([0, 1])
        #plt.ylabel('Precision')
        #plt.xlabel('Recall') 
    
        for method_name in methods_to_plot:
            #print(method_name)
            seq_id_to_windowsums = {}
            seq_id_to_scores=method_to_seq_id_to_scores[method_name]
            for seq_id,scores in seq_id_to_scores.items():
                #cumsum = np.array([0]+list(np.cumsum(np.sum(scores,axis=-1))))
                cumsum = np.array([0]+list(np.cumsum(scores)))
                windowsums = cumsum[motif_len:]-cumsum[:-motif_len]
                seq_id_to_windowsums[seq_id] = windowsums
            loc_scores = [seq_id_to_windowsums[seq_id][pos]
                          for (seq_id, pos) in all_locs]
            auroc = roc_auc_score(y_true=loc_labels,
                                  y_score=loc_scores)
            auprc = average_precision_score(y_true=loc_labels,
                                            y_score=loc_scores)
            #print("auROC: " + str(auroc))
            #print("auPRC: " + str(auprc))
            #print("Hits in pos set: " + str(motif_id_to_pos_locs[motif_id]))
            motif_info.update({motif_id:[auroc,auprc,motif_id_to_pos_locs[motif_id]]})
            """plt.subplot(122)
            fpr, tpr, rocthresholds = roc_curve(
                loc_labels, loc_scores, pos_label=1)
            plt.plot(fpr, tpr,
                     label = method_name_nicename[method_name]+(' AUC = %0.2f' % (100*auroc))+"%",
                     linewidth=2,
                     **method_name_to_plot_style[method_name])
            plt.legend(loc = 'lower right', fontsize='xx-small')

            plt.subplot(121)
            precision, recall, thresholds = precision_recall_curve(
                loc_labels, loc_scores, pos_label=1)
            plt.plot(precision, recall,
                     label = method_name_nicename[method_name]+(' AUC = %0.2f' % (100*auprc))+"%",
                     linewidth=2,
                     **method_name_to_plot_style[method_name])
            plt.legend(loc = 'top right', fontsize='xx-small')

        plt.show()"""

sorted_keys=(sorted(motif_info, key=lambda x: motif_info[x][1]))
for key in sorted_keys[::-1]:
    print(key+": "+str(motif_info[key]))

Number of motifs is 75
CCACYAGRKGGC: [0.9764731034102485, 0.71748083630887, 353.0]
CACTAGRGGG: [0.9797909473008104, 0.6752914006556534, 278.0]
CCCCTAGTGGCC: [0.9282952038681368, 0.49674051902310123, 134.0]
CCRCTAGGKG: [0.9590597227031163, 0.4905623286192296, 146.0]
ATGACTCA: [0.9798172683967634, 0.4394746579108471, 446.0]
ATGACTCATC: [0.9721418604735722, 0.3740816771512532, 431.0]
TGASTCAB: [0.9567088907017632, 0.35354617753371725, 657.0]
NNATGASTCATN: [0.9756676563850748, 0.3309039873363139, 407.0]
CYAGGGGGCGCT: [0.8903750669525721, 0.3085971722448273, 116.0]
STTAVTCABH: [0.8964442149002603, 0.24396702060645015, 786.0]
CTAGCGGC: [0.852106000888972, 0.17403580391453563, 525.0]
CAGTCATK: [0.8249274194311456, 0.13356417921397146, 1649.0]
AGAGGGCGCT: [0.7426374757782344, 0.11598061641254896, 751.0]
CGCGCBCT: [0.7066155729017861, 0.06698861423290592, 695.0]
SCYYTARR: [0.7747514998539695, 0.04572896736237263, 1287.0]
VACWTTCC: [0.7589078781955958, 0.024529873314344405, 1244.0]
TSGGAAMG: [0.

# Sanity Check

#### Extract base pair sequences

In [ ]:
shuffled_motifs={'SAGATAAV', 'NVCTTTYACA', 'NTRTCTAGCK', 'NNATGASTCATN', 'DCTGTRAAARVN', 'STTCYDGGAA', 'WGAAAAGCAG', 'CGCGGCGC', 'CAGCCAAANV', 'RCCAATCG', 'NBNTTATCTG', 'GGCGSCGSCG', 'CMYCTRGTGG', 'SNAAACMGCH', 'GGGGGAGGGGSN', 'AGGAAACG', 'CAGCCAMA', 'GNTCTCGCGAGA', 'AAAAGCHGNCTN', 'GCCGCTAG', 'AACAGCAG', 'VAGRYGGCGSCN', 'GCCCCGCCCC', 'NBTGTYTAGCTG', 'ACCAGAAG', 'YGTTTCCHNN', 'HRAATGGAAT', 'CCTTCCCC', 'TGTCTAGC', 'GCTTTGCAAA', 'AAAGCCDC', 'AAACCAGA', 'AAAGSCNNNN', 'CCCCNCCC', 'YGTTTACA', 'RMTGACWG', 'RKCWGTAAAR', 'YTTCTGGTTT', 'GCTCCTCC', 'ATGACWDCRN', 'GSGGGGGSGGSG', 'ATGACTCA', 'GCCMYCTRGTGG', 'ACTTCCTB', 'NWGATAANVNNN', 'NNGCGCVKSCGC', 'AGATAGGRNNNN', 'RCTTCCTBYYNN', 'CCGCGCGGCG', 'CTTTCACA', 'NVMGCNVGCG', 'BYTGCTGTTW', 'GTGTGTGT', 'GATGAGTCAT', 'RCCAATSVSNNN', 'GTGACDTC', 'HGTSACTTVD', 'GCTCCWCCCG', 'CSGCGGCG', 'NHGCAGAAAARA', 'NTSTGGCTKBHD', 'CGCCCTCT', 'CTTTCMCAGAAG', 'NDGTCACRTGAC', 'TTTACARWCCYT', 'RGAGGAAGYG', 'NNDGHAGTCACT', 'GWGTGTGTGT', 'NNNARACAGCAR', 'AGCGCGCG', 'NCMVCTCCCYCN', 'NGBCCCGCGVGV', 'GWGTGTGTGTGT', 'DGCTGTTW', 'NGGGYGGRGCSR'}
original_motifs={'AAAGCSDC', 'HGGCCCCGCCCC', 'ACCAATCG', 'NNDACNGCNS', 'TTGGWGAACCTT', 'CCAATCAG', 'CCCCTCCCCCNC', 'ACTTCCTSYTBN', 'CAGCCAAA', 'AGGGTTTGTAAA', 'ASMMAAACAS', 'STTATCWG', 'GGCGCGGGCGCG', 'GAGTGTGTGTGT', 'YTGTCAGB', 'RTGACGTCAYCS', 'TAAAAGCAGGCT', 'CMYCTAGYGG', 'ACTACRNYTCCC', 'GGCGGCGGCG', 'HKCGCGCG', 'CGCAAGATTTAT', 'TGBAAACG', 'AGTGACCTCTAG', 'VTCATGTGAC', 'TGTGAAAG', 'GCCMCGCC', 'DGTGACGTCA', 'CGCCGCCG', 'CACYAGRGGG', 'TGTCTAGC', 'GCWGHMAAAAMV', 'BYTSTGGTTT', 'CGCGCNNGCG', 'ATTATACGCTAA', 'RMTCTCGCGAGA', 'RRCCAATCRG', 'CTAGCGGC', 'CWGATAAGANNN', 'TBTRKCTAGCTV', 'GGGHGGAGCC', 'CTCCGCCC', 'GCCCCGCCCC', 'VTTCYNGGAA', 'ATGACTCA', 'TCGAATGGAATC', 'CYCTGTMAAA', 'KGCKGTTT', 'RASMGGAAGT', 'CTGCGCATGCGC', 'CCGCGCGGCG', 'DACAGCWG', 'NGVTGASTCATC', 'GATGAGTCAT', 'AAGATGGCGGCS', 'RCCAATCAGMDB', 'TGGTCTAGCGGT', 'CTTATCTSNN', 'GRGGRAGT', 'CCCCCCCC', 'GCCMCCTAGTGG', 'CCMCKCCCMC', 'CGCATGCG', 'CGCCCTCT', 'TKAGCATGCT', 'WGATAAVS', 'KVKCGCGVGA', 'RYCAYRTGRYHN', 'NBCTTATCTS', 'GTGACTWC', 'GTRTCTAGCT', 'CTTCCGGT', 'SGWTYGTRAA', 'AGCGCCCCCT'}
print ("Intersects:", shuffled_motifs.intersection(original_motifs))

In [ ]:
import gzip
import re
from collections import OrderedDict
def load_sequences_from_bedfile(seqfile):
    seqs = OrderedDict()
    fp = gzip.open(seqfile, "rb")
    print("#Loading " + seqfile + " ...")
    for line in fp:
        (label, sequence)=line.split()
        seqs[label]=sequence
    fp.close()
    print("#Loaded " + str(len(seqs.keys())) + " sequences from " + seqfile)
    return seqs

In [ ]:
def rename(label):
    match=re.match('.*_(chr.*)$',label)
    if match:
        return match.group(1)
    else:
        return ""

In [ ]:
data_filename_positive = "/users/eprakash/projects/benchmarking/newdata/SPI1_in_K562/SPI1.pos.summits.implanted.bed.gz"
data_filename_negative = "/users/eprakash/projects/benchmarking/newdata/SPI1_in_K562/SPI1.neg.summits.implanted.bed.gz"
labeled_sequences = load_sequences_from_bedfile(data_filename_positive)
neg_seqs = load_sequences_from_bedfile(data_filename_negative)
labeled_sequences.update(neg_seqs)
sequences={}
for (label,seq) in labeled_sequences.items():
    sequences.update({rename(label):seq})

#### Select random 5 sequences for motif

In [ ]:
from random import *
motif='CTTCCTTYCT'
motif_seqs=[value[0] for value in motif_id_to_hit_locations[motif]]
random_indices=[randint(0,len(motif_seqs)) for i in range(0,5)]
print (random_indices)
selected_seqs=[motif_seqs[i] for i in random_indices]
print (selected_seqs)

#### Onehot encode

In [ ]:
def one_hot_encode_along_channel_axis(sequence):
    to_return = np.zeros((len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=sequence, one_hot_axis=1)
    return to_return

def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1
#sequences = sequences[-5000:]            
onehot_data = np.array([one_hot_encode_along_channel_axis(sequences[select_seq]) for select_seq in selected_seqs])

In [ ]:
print (onehot_data.shape)

#### Get relevant scores

In [ ]:
selected_relevant_indices_list=[list(labels).index(seq) for seq in selected_seqs]
selected_rescale_conv_scores=get_relevant_scores(selected_relevant_indices_list, original_rescale_conv_scores, seq_len)
print (selected_rescale_conv_scores.shape)

In [ ]:
def split_sequence_by_motif_name(motif_name, list_of_dicts):
    list_with_motif = list()
    list_without_motif = list()
    for one_motif_occurrence in list_of_dicts:
        full_motif = one_motif_occurrence['motif']
        essential_motif = re.match('(\d+)-(\w+)-(\d+)',full_motif).group(2)
        if essential_motif == motif_name:
            list_with_motif.append(one_motif_occurrence)
        else:
            list_without_motif.append(one_motif_occurrence)
    return (list_with_motif, list_without_motif)

#### Visualization

In [ ]:
%matplotlib inline
from deeplift.visualization import viz_sequence
for i in range(0,len(selected_seqs)):
    scores=selected_grad_times_input_scores[i]
    seq_id=selected_seqs[i]
    print(seq_id)
    (list_with_motif, list_without_motif)=split_sequence_by_motif_name(motif, motif_matches[seq_id])
    viz_sequence.plot_weights(onehot_data[i]*scores[:, None], subticks_frequency=10, highlight={'blue':[(seq_info['begin'],seq_info['end']) for seq_info in list_with_motif]})
    viz_sequence.plot_weights(onehot_data[i]*scores[:, None], subticks_frequency=10, highlight={'red':[(seq_info['begin'],seq_info['end']) for seq_info in list_without_motif]})

In [ ]:
pwm = np.array([[0.036,0.170,0.033,0.761],[0.001,0.635,0.125,0.239],[0.031,0.128,0.004,0.837],[0.001,0.676,0.166,0.157],
                [0.058,0.113,0.040,0.789],[0.007,0.676,0.207,0.110],[0.007,0.125,0.001,0.867],[0.001,0.660,0.206,0.133],
                [0.005,0.186,0.001,0.808],[0.001,0.607,0.241,0.151],[0.054,0.231,0.006,0.709],[0.025,0.540,0.217,0.218]])
print (pwm.shape)

In [ ]:
%matplotlib inline
from deeplift.visualization import viz_sequence
viz_sequence.plot_weights(pwm, subticks_frequency=10)